# Pre-pro

In [40]:
!pip install wandb
!pip install transformers
!pip install sentence_transformers
!pip install pipeline

In [41]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import torch
from torch.nn.functional import softmax
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import LongformerTokenizer, LongformerForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification

from math import *

import sys
sys.path.append('models')
import matplotlib.pyplot as plt
%matplotlib inline
import wandb
from wandb.keras import WandbCallback
import json
from google.colab import drive
drive.mount('/content/gdrive')
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, get_linear_schedule_with_warmup

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [70]:
from sklearn.metrics import confusion_matrix
import plotly.express as px
def show_conf_matrix(y_test, pred, labels_name):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  data = cf
  fig = px.imshow(data,
                  labels={'y' : "Vrai Catégorie", 'x' :"Catégorie prédite", 'color' : "Productivity"},
                  x=labels_name,
                  y=labels_name
                )
  fig.update_xaxes(side="top")
  fig.show()

In [43]:
def function_reu(x):
  try:
    return dic[x]
  except:
    return ' '
def function_test(x):
  try:
    return dictionnary[(x+ '.mp3')]
  except:
    return 'do not have'

In [44]:
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans0.txt', encoding='utf-16')
text0 = file.read()
d = json.loads(text0)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans1.txt', encoding='utf-16')
text1 = file.read()
d1 = json.loads(text1)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans2.txt', encoding='utf-16')
text2 = file.read()
d2 = json.loads(text2)
file = open('/content/gdrive/MyDrive/Projet_Multimedia/download/Texte/trans_rest.txt', encoding='utf-16')
text3 = file.read()
d3 = json.loads(text3)
dictionnary = d | d1
dictionnary = dictionnary | d2
dictionnary = dictionnary | d3
len(dictionnary)
descr = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/Transcription_wav.csv')
name_video = descr['name_vid']
contenue = descr['contenue']
dic = {}
vid_name_set = set()
for vid_name, cont in zip(list(name_video), list(contenue)):
  if not 'artificial intelligence' in cont.lower():
    if not vid_name in dic.keys():
      dic[vid_name] = cont
    else:
      dic[vid_name] += cont
  else:
    vid_name_set.add(vid_name)
    print(vid_name)
descr['descr'] = descr['name_vid'].apply(lambda x : function_reu(x))
descr = descr.drop_duplicates(subset='name_vid')
dt = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVT.csv')
dt['name'] = dt['video'].apply(lambda x : x[5:-4])
dt['transcription'] = dt['name'].apply(lambda x : function_test(x))
dt = dt.drop('video', axis=1)
dt = dt[dt.transcription != 'do not have']
intersection = pd.merge(dt, descr, left_on='name', right_on='name_vid')
intersection = intersection.drop('label', axis = 1)
intersection = intersection.drop('Unnamed: 0', axis = 1)
intersection = intersection.drop('name_audio', axis = 1)
intersection = intersection.drop('id_audio', axis = 1)
intersection = intersection.drop('name', axis = 1)
intersection = intersection.drop('name_vid', axis = 1)
intersection = intersection.drop('contenue', axis = 1)
intersection['total'] = intersection['text'] + intersection['descr'] + intersection['transcription']
print(len(intersection), len(dt), len(descr))

-eOrNdBpGMv8
1551 2085 1878


In [45]:
import pickle as pk
with open('/content/gdrive/MyDrive/Projet_Multimedia/download/raw_captions_combined_filtered.pkl', "rb") as fp:
    dict_caption= pk.load(fp)
def liste_strings_concat(x):
  strings = ''
  for elem in x:
    for world in elem:
      strings += world + ' '
  return strings
liste_strings_concat(dict_caption['video-__5k7e0f3r4'])
df = pd.read_csv('/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/LVD.csv')
df['text'] = df['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))

In [46]:
df_test = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Test_SurSampling1.csv")
df_train = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Train_SurSampling1.csv")
df_valid = pd.read_csv("/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/DataSets/Val_SurSampling1.csv")
len(df_test), len(df_train), len(df_valid)

(640, 1909, 639)

In [47]:
df_train['text'] = df_train['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))
df_test['text'] = df_test['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))
df_valid['text'] = df_valid['name'].apply(lambda x : liste_strings_concat(dict_caption['video' + x ]))

In [48]:
df['label'].value_counts()

Education                732
People & Blogs           340
Entertainment            301
Film & Animation         286
Howto & Style            137
Comedy                    95
Nonprofits & Activism     95
Science & Technology      65
Pets & Animals            53
News & Politics           52
Music                     39
Travel & Events           32
Sports                    21
Gaming                    17
Autos & Vehicles          15
Name: label, dtype: int64

In [49]:
df

,name,label,duree,text
0,-KHQk1_Vq69E,Autos & Vehicles,21,in this video you will learn about a bar graph...
1,-mG4Y2Snygfk,People & Blogs,60,when the screen comes on we see a notebook at ...
2,-OdEfx8up7wM,People & Blogs,69,Jack in the dog looking child grimaces while h...
3,-Opbfh4wNu7Q,Entertainment,61,Captain America Ironman and Thor walk towards ...
4,-A5340xbbQJU,Sports,299,the Swing Shot is fired firing the basketball ...
...,...,...,...,...
2275,-ymc30meWzfg,Education,79,Professor takes the blood of a patient Tiffany...
2276,-h7HGWRTGqyg,Education,164,a blindfold adult wearing green sits next to a...
2277,-XVRLTfN1WYw,Howto & Style,90,add some whipped cream caramel waffle cone a m...
2278,-jTtH7Fq1trw,Travel & Events,185,located in the Sugar Factory Fashion Show in L...


# DP

## DistilBERT

In [78]:
# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.15, random_state=2)
train_df, valid_df = train_test_split(train_df, test_size=0.10, random_state=2)


In [79]:
len(train_df), len(valid_df), len(test_df)

(1744, 194, 342)

In [80]:
train_df['label'].value_counts(), test_df['label'].value_counts()

(Education                553
 People & Blogs           252
 Entertainment            232
 Film & Animation         223
 Howto & Style            114
 Nonprofits & Activism     74
 Comedy                    72
 Science & Technology      52
 Pets & Animals            43
 News & Politics           38
 Music                     29
 Travel & Events           21
 Sports                    16
 Gaming                    14
 Autos & Vehicles          11
 Name: label, dtype: int64,
 Education                109
 People & Blogs            60
 Entertainment             45
 Film & Animation          38
 Comedy                    16
 Nonprofits & Activism     14
 Howto & Style             12
 News & Politics            9
 Pets & Animals             9
 Science & Technology       8
 Travel & Events            8
 Music                      5
 Sports                     4
 Gaming                     3
 Autos & Vehicles           2
 Name: label, dtype: int64)

In [84]:
train_df, test_df, valid_df = df_train, df_test, df_valid
train_df['label'] = df_train['label'].astype('category')
test_df['label'] = df_test['label'].astype('category')
valid_df['label'] = df_valid['label'].astype('category')

In [85]:
len(train_df), len(valid_df), len(test_df)

(1909, 639, 640)

In [86]:
train_df['label'].value_counts(), test_df['label'].value_counts()

(Education                439
 People & Blogs           204
 Entertainment            181
 Film & Animation         172
 Autos & Vehicles          83
 Comedy                    83
 Gaming                    83
 Howto & Style             83
 Music                     83
 News & Politics           83
 Nonprofits & Activism     83
 Pets & Animals            83
 Science & Technology      83
 Sports                    83
 Travel & Events           83
 Name: label, dtype: int64,
 Education                147
 People & Blogs            68
 Entertainment             60
 Film & Animation          57
 Autos & Vehicles          28
 Comedy                    28
 Gaming                    28
 Howto & Style             28
 Music                     28
 News & Politics           28
 Nonprofits & Activism     28
 Pets & Animals            28
 Science & Technology      28
 Sports                    28
 Travel & Events           28
 Name: label, dtype: int64)

In [101]:
# Charger le tokenizer et le modèle pré-entraîné DistilBERT plus petit
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['label'].cat.categories)).to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
label_name = 'text' #what we changed when tried other types of text -> intersection of text (concatenation)

In [103]:
# Tokeniser les textes
train_tokens = tokenizer.batch_encode_plus(train_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')
valid_tokens = tokenizer.batch_encode_plus(valid_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')
test_tokens = tokenizer.batch_encode_plus(test_df[label_name].tolist(), padding=True, truncation=True, return_tensors='pt')

In [104]:
# Créer des jeux de données PyTorch
train_dataset = TensorDataset(train_tokens['input_ids'].to('cuda'),train_tokens['attention_mask'].to('cuda'),torch.tensor(train_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))
valid_dataset = TensorDataset(valid_tokens['input_ids'].to('cuda'),valid_tokens['attention_mask'].to('cuda'),torch.tensor(valid_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))
test_dataset = TensorDataset(test_tokens['input_ids'].to('cuda'), test_tokens['attention_mask'].to('cuda'), torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long).to('cuda'))

In [105]:
# Définir le DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_valid = 0.0, 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()

    for batch in valid_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_valid += loss.item()

    average_loss_train, average_loss_valid = total_loss_train / len(train_loader), total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {average_loss_train}, val_loss: {average_loss_valid}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epoch 1/5, loss: 2.255866608859106, val_loss: 1.9721949219703674
Epoch 2/5, loss: 1.4332818648306396, val_loss: 1.8275840796530247
Epoch 3/5, loss: 0.8163646644129414, val_loss: 1.845581129938364
Epoch 4/5, loss: 0.46159358827638824, val_loss: 2.0316098921000956
Epoch 5/5, loss: 0.24701171907670327, val_loss: 2.2013230569660664


In [ ]:
# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 1

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_valid = 0.0, 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()

    for batch in valid_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_valid += loss.item()

    average_loss_train, average_loss_valid = total_loss_train / len(train_loader), total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {average_loss_train}, val_loss: {average_loss_valid}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epoch 1/1, loss: 0.4207407680783168, val_loss: 1.8438323969776567


### Resultat label normaux

In [66]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_preds_proba.extend(logits)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

Accuracy: 0.6052631578947368


In [68]:
dic = {}
for count, elem in enumerate(torch.tensor(test_df['label'].cat.codes.tolist())):
  if not (elem in dic.keys()):
    dic[elem.item()] = list(test_df['label'])[count]
dic

{2: 'Education',
 10: 'People & Blogs',
 4: 'Film & Animation',
 3: 'Entertainment',
 11: 'Pets & Animals',
 8: 'News & Politics',
 12: 'Science & Technology',
 6: 'Howto & Style',
 9: 'Nonprofits & Activism',
 13: 'Sports',
 14: 'Travel & Events',
 0: 'Autos & Vehicles',
 7: 'Music',
 1: 'Comedy',
 5: 'Gaming'}

In [69]:
list_cat = []
for i in np.sort(list(set(all_preds))):
  list_cat.append(dic[i])

In [71]:
print(len(set(all_preds)), len(set(test_df['label'])))
show_conf_matrix(all_preds, torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long), list_cat)

13 15


In [74]:
for elem in range (15):
  if elem not in all_preds:
    print('there was no prediction of', dic[elem], 'there was ', len(train_df[train_df.label == dic[elem]]), 'elements in the train, and it was supposed to find ', len(test_df[test_df.label == dic[elem]]))

there was no prediction of Autos & Vehicles there was  11 elements in the train, and it was supposed to find  2
there was no prediction of Gaming there was  14 elements in the train, and it was supposed to find  3


### resultat label avec Sur/Sub sampling

In [106]:
# Évaluer le modèle
model.eval()
all_preds = []
all_preds_proba = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        all_preds_proba.extend(logits)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_sur_gpu.pth')

Accuracy: 0.4734375


(best result was 47.7%)

In [107]:
dic = {}
for count, elem in enumerate(torch.tensor(test_df['label'].cat.codes.tolist())):
  if not (elem in dic.keys()):
    dic[elem.item()] = list(test_df['label'])[count]
dic

{4: 'Film & Animation',
 2: 'Education',
 10: 'People & Blogs',
 6: 'Howto & Style',
 13: 'Sports',
 3: 'Entertainment',
 0: 'Autos & Vehicles',
 9: 'Nonprofits & Activism',
 14: 'Travel & Events',
 12: 'Science & Technology',
 5: 'Gaming',
 11: 'Pets & Animals',
 1: 'Comedy',
 8: 'News & Politics',
 7: 'Music'}

In [108]:
list_cat = []
for i in np.sort(list(set(all_preds))):
  list_cat.append(dic[i])

In [109]:
print(len(set(all_preds)), len(set(test_df['label'])))
show_conf_matrix(all_preds, torch.tensor(test_df['label'].cat.codes.tolist(), dtype=torch.long), list_cat)

14 15


In [110]:
for elem in range (15):
  if elem not in all_preds:
    print('there was no prediction of', dic[elem], 'there was ', len(train_df[train_df.label == dic[elem]]), 'elements in the train, and it was supposed to find ', len(test_df[test_df.label == dic[elem]]))

there was no prediction of Autos & Vehicles there was  83 elements in the train, and it was supposed to find  28


In [112]:
reverse_dict = {v: k for k, v in dic.items()}

In [122]:
index = np.where(test_df['label'] == 'Autos & Vehicles')[0]
for elem in set(np.array(all_preds)[index]):
  print(dic[elem])

Entertainment


Every autos & vehicles went into Entertainement, when we looked at the video this makes sense.

### DP

In [123]:
# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, input_size, num_labels):
        super(ClassificationModel, self).__init__()
        self.fc = torch.nn.Linear(input_size, 512)
        self.r = torch.nn.ReLU()
        self.m = torch.nn.Dropout(p=0.3)
        self.fc1 = torch.nn.Linear(512, 512)
        self.fc2 = torch.nn.Linear(512, num_labels)

    def forward(self, x):
      x = self.fc(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc1(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc2(x)
      x = self.r(x)
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [124]:
model = ClassificationModel()

TypeError: ignored

In [ ]:
# Définir le DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_valid = 0.0, 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()

    for batch in valid_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_valid += loss.item()

    average_loss_train, average_loss_valid = total_loss_train / len(train_loader), total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {average_loss_train}, val_loss: {average_loss_valid}')

## SentenceBERT

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sentence_transformers import SentenceTransformer

# Charger votre dataframe
# Supposons que votre dataframe a deux colonnes : 'Text' pour les phrases et 'Label' pour les labels

# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['transcription'], df['label'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# Charger le modèle Sentence-Transformers
sentencebert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Encoder les textes avec Sentence-Transformers
train_embeddings = sentencebert_model.encode(X_train.tolist(), convert_to_tensor=True)
test_embeddings  = sentencebert_model.encode(X_test.tolist(), convert_to_tensor=True)
valid_embeddings  = sentencebert_model.encode(X_val.tolist(), convert_to_tensor=True)

# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, input_size, num_labels):
        super(ClassificationModel, self).__init__()
        self.fc = torch.nn.Linear(input_size, 512)
        self.r = torch.nn.ReLU()
        self.m = torch.nn.Dropout(p=0.3)
        self.fc1 = torch.nn.Linear(512, 512)
        self.fc2 = torch.nn.Linear(512, num_labels)

    def forward(self, x):
      x = self.fc(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc1(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc2(x)
      x = self.r(x)
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'

KeyError: ignored

In [ ]:
model = ClassificationModel(input_size=train_embeddings.size(-1), num_labels=len(df['label'].cat.categories)).to(device)

# Définir le DataLoader
batch_size   = 8
train_loader = DataLoader(TensorDataset(train_embeddings.to(device), torch.tensor(y_train.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_embeddings.to(device) , torch.tensor(y_test.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)
valid_loader  = DataLoader(TensorDataset(valid_embeddings.to(device) , torch.tensor(y_val.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 30

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_loss_valid = 0.0

    for batch in train_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    for batch in valid_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss_valid += loss.item()
    average_loss_valid = total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Val_loss: {average_loss_valid}')

# Évaluer le modèle
model.eval()
all_preds  = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        outputs = model(embeddings)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèle
torch.save(model.state_dict(), 'Model_with_sentence_transformers.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30, Loss: 2.661566908596552, Val_loss: 2.582019847736024
Epoch 2/30, Loss: 2.4507568684237744, Val_loss: 2.287903066267047
Epoch 3/30, Loss: 2.190790099707263, Val_loss: 2.1252623440926537
Epoch 4/30, Loss: 2.107034396706966, Val_loss: 2.0776520277324475
Epoch 5/30, Loss: 2.0646963279846817, Val_loss: 2.045655511973197
Epoch 6/30, Loss: 2.0276157367996306, Val_loss: 2.027925788310536
Epoch 7/30, Loss: 1.9878539892665126, Val_loss: 1.989856765981306
Epoch 8/30, Loss: 1.9527478545729877, Val_loss: 1.9632588060278642
Epoch 9/30, Loss: 1.917221889161227, Val_loss: 1.949873832234165
Epoch 10/30, Loss: 1.8768512951700311, Val_loss: 1.9154026759298224
Epoch 11/30, Loss: 1.849576587565461, Val_loss: 1.8851400530129148
Epoch 12/30, Loss: 1.809706162988094, Val_loss: 1.8838427547822918
Epoch 13/30, Loss: 1.7837685353574697, Val_loss: 1.862721742245189
Epoch 14/30, Loss: 1.756300625745316, Val_loss: 1.8473257181937235
Epoch 15/30, Loss: 1.7354567650465937, Val_loss: 1.8346936723642182
Epo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sentence_transformers import SentenceTransformer

# Charger votre dataframe
# Supposons que votre dataframe a deux colonnes : 'Text' pour les phrases et 'Label' pour les labels

# Convertir la colonne 'Label' en type 'category'
df['label'] = df['label'].astype('category')

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# Charger le modèle Sentence-Transformers
sentencebert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Encoder les textes avec Sentence-Transformers
train_embeddings = sentencebert_model.encode(X_train.tolist(), convert_to_tensor=True)
test_embeddings  = sentencebert_model.encode(X_test.tolist(), convert_to_tensor=True)
valid_embeddings  = sentencebert_model.encode(X_val.tolist(), convert_to_tensor=True)

# Définir le modèle de classification
class ClassificationModel(torch.nn.Module):
    def __init__(self, input_size, num_labels):
        super(ClassificationModel, self).__init__()
        self.fc = torch.nn.Linear(input_size, 512)
        self.r = torch.nn.ReLU()
        self.m = torch.nn.Dropout(p=0.3)
        self.fc1 = torch.nn.Linear(512, 512)
        self.fc2 = torch.nn.Linear(512, num_labels)

    def forward(self, x):
      x = self.fc(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc1(x)
      x = self.r(x)
      x = self.m(x)
      x = self.fc2(x)
      return x

# Initialiser le modèle de classification
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ClassificationModel(input_size=train_embeddings.size(-1), num_labels=len(df['label'].cat.categories)).to(device)

# Définir le DataLoader
batch_size   = 8
train_loader = DataLoader(TensorDataset(train_embeddings.to(device), torch.tensor(y_train.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_embeddings.to(device) , torch.tensor(y_test.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)
valid_loader  = DataLoader(TensorDataset(valid_embeddings.to(device) , torch.tensor(y_val.cat.codes.tolist(), dtype=torch.long).to(device)), batch_size=batch_size, shuffle=False)

# Définir l'optimiseur
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 30

# Entraîner le modèle
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_loss_valid = 0.0

    for batch in train_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    for batch in valid_loader:
        embeddings, labels = batch
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        total_loss_valid += loss.item()
    average_loss_valid = total_loss_valid / len(valid_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Val_loss: {average_loss_valid}')

# Évaluer le modèle
model.eval()
all_preds  = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        outputs = model(embeddings)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# Calculer la précision
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')

# Sauvegarder le modèleSentenceTransformer
torch.save(model.state_dict(), 'Model_with_sentence_transformers.pth')

# ML sur les embeddings ci-dessus

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pipeline

In [ ]:
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(sentencebert_model.encode(X_train.tolist()),y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
tfsvmpred = clf.predict(sentencebert_model.encode(X_test.tolist()))
accuracy_score(y_test,tfsvmpred)

0.5142857142857142

# ML

In [ ]:
train_df, test_df = train_test_split(dt, test_size=0.2, random_state=42)
X_train, y_train = train_df['transcription'], train_df['label']
X_test, y_test = test_df['transcription'], test_df['label']

In [ ]:
emb_train = sentencebert_model.encode(X_train.tolist())
emb_test = sentencebert_model.encode(X_test.tolist())

In [ ]:
X_train = torch.cat([train_tokens['input_ids'], train_tokens['attention_mask']], 1)
X_test = torch.cat([test_tokens['input_ids'], test_tokens['attention_mask']], 1)
y_train, y_test = train_df['label_encoded'], test_df['label_encoded']

In [ ]:
X_train

tensor([[  101,  2550,  2011,  ...,     0,     0,     0],
        [  101,  4918,  4138,  ...,     0,     0,     0],
        [  101,  4639,  2450,  ...,     0,     0,     0],
        ...,
        [  101,  7760,  2013,  ..., 14026,  1996,   102],
        [  101,  8194,  1998,  ...,  1012,  2123,   102],
        [  101,  2510,  4946,  ...,  2057,  2288,   102]])

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train,y_train)
tfsvmpred = clf.predict(X_test)
accuracy_score(y_test,tfsvmpred)

0.2861736334405145

In [ ]:
predovr = OneVsRestClassifier(SVC(random_state=0)).fit(X_train, y_train).predict(X_test)
accuracy_score(y_test,predovr)

0.29260450160771706

In [ ]:
clf = RandomForestClassifier(n_estimators = 500, max_depth=12, random_state=0)
clf.fit(X_train, y_train)
accuracy_score(clf.predict(X_test),y_test)

0.2797427652733119

In [ ]:
clf = GradientBoostingClassifier(random_state=0, n_estimators = 50).fit(emb_train, y_train)
clf.fit(emb_train, y_train)
accuracy_score(clf.predict(emb_test),y_test)



1.   intersection : [47.9, 43.4 , 41.2, 40.5]
2.   dt:transcription : [43.4, 43.4 , 41.2, 40.5]



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
calibrated_forest = CalibratedClassifierCV(estimator=RandomForestClassifier(n_estimators=10))
param_grid = {'estimator__max_depth': [2, 4, 6, 8]}
search = GridSearchCV(calibrated_forest, param_grid, cv=5)
search.fit(emb_train, y_train)
GridSearchCV(cv=5,
             estimator=CalibratedClassifierCV(...),
             param_grid={'estimator__max_depth': [2, 4, 6, 8]})

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

In [ ]:
y_test

941     2
396     9
1071    9
1282    4
491     2
       ..
87      8
1112    0
330     2
494     3
496     7
Name: label_encoded, Length: 311, dtype: category
Categories (15, int64): [0, 1, 2, 3, ..., 11, 12, 13, 14]